In [1]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader

In [2]:
list_dirs = [
    "Healthy",
    "Motor_1_Steady_state_error",
    "Motor_1_Stuck",
    "Motor_2_Steady_state_error",
    "Motor_2_Stuck",
    "Motor_3_Steady_state_error",
    "Motor_3_Stuck",
    "Motor_4_Steady_state_error",
    "Motor_4_Stuck"
    ]

trainpath = "set_csv/training_set/"
augmentedpath = "set_csv/augmented/"
testpath = "set_csv/testing_set/"

paths = [trainpath, augmentedpath, testpath]

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [10]:
trainlist = []
augmentedlist = []
testlist = []

tensors = []

for path in paths:
    liste = []
    for dir in list_dirs:
        categorylist = []
        with os.scandir(path + dir) as it:
            for file in it:
                if path == augmentedpath:
                    dataframe = np.array(pd.read_csv(file.path))[:,2:]
                else:
                   dataframe = np.array(pd.read_csv(file.path))[:,1:]
                tensor = torch.tensor(dataframe)
                categorylist.append(tensor)
        categorytensor = torch.stack(categorylist)
        liste.append(categorytensor)
    tensors.append(torch.stack(liste))

train_tensor, augmented_tensor, test_tensor = tensors


print(train_tensor.shape, augmented_tensor.shape, test_tensor.shape)

torch.Size([9, 400, 1000, 6]) torch.Size([9, 120, 1000, 6]) torch.Size([9, 10, 1000, 6])


In [ ]:
class DataSet(Dataset):
    def __init__(self, tens, device):
        self.device = device
        self.data = tens.to(torch.float32)
        self.data = (self.data[:,:,:,:3] - self.data[:,:,:,3:]).to(self.device)
            
        self.label = []
        for n in range(9):
            labels = n * torch.ones(len(self.data[0])).to(torch.int16)
            self.label.append(labels)
        self.label = torch.flatten(torch.stack(self.label)).to(device)
        self.data = torch.flatten(self.data, end_dim=1)
        assert len(self.label) == len(self.data)
        print(self.data.shape)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index], self.label[index]
    
train_set = DataSet(train_tensor, device)
augmented_set = DataSet(augmented_tensor, device)
test_set = DataSet(test_tensor, device)

torch.Size([3600, 1000, 3])
torch.Size([90, 1000, 3])


In [ ]:
train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
augmented_loader = DataLoader(augmented_set, batch_size=20, shuffle=True)
test_loader = DataLoader(test_set, batch_size=90)

In [6]:
class lstm_model(nn.Module):
    def __init__(self, hidden_size, device):
        super().__init__()
        self.hidden_size = hidden_size
        self.device = device

        self.Lstm = nn.LSTM(3, self.hidden_size, device=self.device, batch_first=True, dropout =.1, num_layers=2)
        self.Linear = nn.Sequential(
            nn.Linear(self.hidden_size, 50),
            nn.BatchNorm1d(50),
            nn.ReLU(),
            nn.Linear(50, 9)
            )
        
        
    def forward(self, x):
        x = self.Lstm(x)[0][:,-1].cpu()
        x = self.Linear(x).to(device)
        return x

model = lstm_model(100, device)

In [7]:
tensors = train_set.data[500:510]
labels = train_set.label[:10]

guesses = model.forward(tensors)
print(torch.argmax(guesses, dim=1))

tensor([1, 6, 1, 8, 4, 8, 4, 6, 6, 1], device='mps:0')


In [ ]:
lr = 1E-4
firstepochs = 100
transferepochs = 50
patience = 10

cost_function = torch.nn.CrossEntropyLoss(reduction = "sum")
optim = torch.optim.Adam(model.parameters(), lr)
scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=30)

In [17]:
for n in range(firstepochs):
    model.train()
    for features, labels in train_loader:
        guesses = model.forward(features)

        cost = cost_function(guesses, labels)
        cost.backward()
        
        optim.step()
        optim.zero_grad()

    scheduler.step()

    print(f"Training. Epoch : {n+1}, cost : {cost}, lr : {scheduler.get_last_lr()[0]}")
    
    with torch.no_grad():
        accuracy = 0
        for features, labels in test_loader:
            guesses = model.forward(features)
            accuracy = torch.sum(torch.argmax(guesses, dim=1) == labels)

    print(f"Validation. Epoch : {n}, accuracy = {accuracy*100/90}% ({accuracy})")

Training. Epoch : 1, cost : 8.355853080749512, lr : 0.0001
Validation. Epoch : 0, accuracy = 28.88888931274414% (26)
Training. Epoch : 2, cost : 9.553357124328613, lr : 0.0001
Validation. Epoch : 1, accuracy = 28.88888931274414% (26)
Training. Epoch : 3, cost : 9.724488258361816, lr : 0.0001
Validation. Epoch : 2, accuracy = 31.11111068725586% (28)
Training. Epoch : 4, cost : 7.507523536682129, lr : 0.0001
Validation. Epoch : 3, accuracy = 28.88888931274414% (26)
Training. Epoch : 5, cost : 11.736729621887207, lr : 0.0001
Validation. Epoch : 4, accuracy = 26.66666603088379% (24)


KeyboardInterrupt: 

In [ ]:
for n in range(transferepochs):
    model.train()
    for features, labels in augmented_loader:
        guesses = model.forward(features)

        cost = cost_function(guesses, labels)
        cost.backward()
        
        optim.step()
        optim.zero_grad()

    scheduler.step()

    print(f"Training. Epoch : {n+1}, cost : {cost}, lr : {scheduler.get_last_lr()[0]}")
    
    with torch.no_grad():
        accuracy = 0
        for features, labels in test_loader:
            guesses = model.forward(features)
            accuracy = torch.sum(torch.argmax(guesses, dim=1) == labels)

    print(f"Validation. Epoch : {n}, accuracy = {accuracy*100/90}% ({accuracy})")

Training. Epoch : 6, cost : 12.693446159362793, lr : 0.001
Validation. Epoch : 6, accuracy = 33.33333206176758% (30)
Training. Epoch : 7, cost : 9.822790145874023, lr : 0.001
Validation. Epoch : 7, accuracy = 42.22222137451172% (38)
Training. Epoch : 8, cost : 12.577727317810059, lr : 0.001
Validation. Epoch : 8, accuracy = 34.44444274902344% (31)
Training. Epoch : 9, cost : 14.596870422363281, lr : 0.001
Validation. Epoch : 9, accuracy = 35.55555725097656% (32)
Training. Epoch : 10, cost : 10.840933799743652, lr : 0.001
Validation. Epoch : 10, accuracy = 34.44444274902344% (31)
Training. Epoch : 11, cost : 6.777524471282959, lr : 0.001
Validation. Epoch : 11, accuracy = 28.88888931274414% (26)
Training. Epoch : 12, cost : 9.85922622680664, lr : 0.001
Validation. Epoch : 12, accuracy = 34.44444274902344% (31)
Training. Epoch : 13, cost : 12.74919605255127, lr : 0.001
Validation. Epoch : 13, accuracy = 31.11111068725586% (28)
